# Translate text without profanities

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/translation_to_specific_language.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to english. We will check whether the translated statement passes the profanity check or not.

## Objective

We want to translate a statement from another languages to English and ensure the translated statement is profanity free.

## Step 0: Setup

In order to run this example, you will need to install `alt-profanity-check` package. You can do so by running the following commands:

In [5]:
! pip install alt-profanity-check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for alt-profanity-check: filename=alt_profanity_check-1.5.0-py3-none-any.whl size=758311 sha256=e0f54f82189ad2c90aeb27cb9239175c71d38606836be9e4762fb64b2e2de0a0
  Stored in directory: /Users/wyatt/Library/Caches/pip/wheels/18/c3/20/637574a9badb43cace85202ca31f49f47e3fe65e076459f3ed
Successfully built alt-profanity-check


## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](/docs/how_to_guides/rail).  We will also show the same RAIL spec in a code-first format using a Pydantic model.

In this RAIL spec, we:

1. Create an `output` schema that returns a single key-value pair. The key should be 'translated_statement', and the value should be the English translation of the given statement. The translated statement should not have any profanity.

First we create our custom Validator:

In [6]:
from profanity_check import predict
from guardrails.validators import (
    Validator,
    register_validator,
    ValidationResult,
    PassResult,
    FailResult,
)


from typing import Dict, Any


@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> Dict:
        prediction = predict([value])
        if prediction[0] == 1:
            return FailResult(
                error_message=f"Value {value} contains profanity language",
                fix_value="",
            )
        return PassResult()

/Users/wyatt/Projects/guardrails/docs/examples/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/wyatt/Projects/guardrails/guardrails/validators/__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(


Next we define our RAIL spec either as XML:

In [7]:
rail_str = """
<rail version="0.1">

<output>
    <string
        name="translated_statement"
        description="Translate the given statement into english language"
        format="is-profanity-free"
        on-fail-is-profanity-free="fix" 
    />
</output>


<prompt>
Translate the given statement into english language:

${statement_to_be_translated}

${gr.complete_json_suffix}
</prompt>

</rail>
"""

Or as a Pydantic model:

In [8]:
from pydantic import BaseModel, Field

prompt = """
Translate the given statement into english language:

${statement_to_be_translated}

${gr.complete_json_suffix}
"""


class Translation(BaseModel):
    translated_statement: str = Field(
        description="Translate the given statement into english language",
        validators=[IsProfanityFree(on_fail="fix")],
    )

/Users/wyatt/Projects/guardrails/guardrails/validator_base.py:460: FutureWarning: Accessing `IsProfanityFree` using
`from guardrails.validators import IsProfanityFree` is deprecated and
support will be removed after version 0.5.x. Please switch to the Guardrails Hub syntax:
`from guardrails.hub import ProfanityFree` for future updates and support.
For additional details, please visit: https://hub.guardrailsai.com/validator/guardrails/profanity_free.

  warn(


!!! note

    In order to ensure the translated statement is profanity free, we use `is-profanity-free` as the validator. This validator uses `profanity_check` package.

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [9]:
import guardrails as gd

from rich import print

From XML:

In [10]:
guard = gd.Guard.from_rail_string(rail_str)

/Users/wyatt/Projects/guardrails/guardrails/prompt/base_prompt.py:59: FutureWarning: Prompt Primitives are moving! To keep the same behaviour, switch from `json` constants to `xml` constants. Example: ${gr.complete_json_suffix} -> ${gr.complete_xml_suffix}
  warn(


Or from our Pydantic model:

In [11]:
guard = gd.Guard.from_pydantic(output_class=Translation, prompt=prompt)

/Users/wyatt/Projects/guardrails/guardrails/validator_base.py:460: FutureWarning: Accessing `IsProfanityFree` using
`from guardrails.validators import IsProfanityFree` is deprecated and
support will be removed after version 0.5.x. Please switch to the Guardrails Hub syntax:
`from guardrails.hub import ProfanityFree` for future updates and support.
For additional details, please visit: https://hub.guardrailsai.com/validator/guardrails/profanity_free.

  warn(
/Users/wyatt/Projects/guardrails/guardrails/prompt/base_prompt.py:59: FutureWarning: Prompt Primitives are moving! To keep the same behaviour, switch from `json` constants to `xml` constants. Example: ${gr.complete_json_suffix} -> ${gr.complete_xml_suffix}
  warn(


We see the prompt that will be sent to the LLM:

In [12]:
print(guard.base_prompt)

/var/folders/8n/8qwytjb11kj_46_w3n2v4jzw0000gn/T/ipykernel_6330/3983563700.py:1: DeprecationWarning: 'Guard.base_prompt' is deprecated and will be removed in versions 0.5.x and beyond. Use 'Guard.history.last.prompt' instead.
  print(guard.base_prompt)


Translate the given statement into english language:

${statement_to_be_translated}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="translated_statement" description="Translate the given statement into english language" 
format="is-profanity-free"/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

Here, `statement_to_be_translated` is the the statement and will be provided by the user at runtime.

## Step 3: Wrap the LLM API call with `Guard`

First, let's try translating a statement that doesn't have any profanity in it.

In [13]:
import openai

raw_llm_response, validated_response, *rest = guard(
    openai.chat.completions.create,
    prompt_params={"statement_to_be_translated": "quesadilla de pollo"},
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

print(f"Validated Output: {validated_response}")

Validated Output: {'translated_statement': 'Chicken quesadilla'}

We can take a look at the output of the LLM and the validated output using the Guard's internal logs:

In [14]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Translate the given statement into english language:                                                    │ │
    │ │                                                                                                         │ │
    │ │ quesadilla de pollo                                                                                     │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="translated_statement" description="Translate the given statement into english         │ │
    │ │ language" format="is-profanity-free"/>                                                                  │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │ You are a helpful assistant, able to express yourself purely through JSON, strictly and precisely       │ │
    │ │ adhering to the provided XML schemas.      

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary). We can see that the output is a dictionary with the correct schema and types.

Next, let's try translating a statement that has profanity in it. We see that the translated statement has been corrected to return an empty string instead of the translated statement.

In [15]:
raw_llm_response, validated_response, *rest = guard(
    openai.chat.completions.create,
    prompt_params={"statement_to_be_translated": "убей себя"},
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

print(f"Validated Output: {validated_response}")

Validated Output: {'translated_statement': ''}

This time around, when we look at the logs, we can see that the output of the LLM was filtered out because it did not pass the profanity check.

In [16]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Translate the given statement into english language:                                                    │ │
    │ │                                                                                                         │ │
    │ │ убей себя                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="translated_statement" description="Translate the given statement into english         │ │
    │ │ language" format="is-profanity-free"/>                                                                  │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │ You are a helpful assistant, able to express yourself purely through JSON, strictly and precisely       │ │
    │ │ adhering to the provided XML schemas.      